In [19]:
%matplotlib inline
import sys
sys.path.append("..") # Adds the module to path

# deeptrack.properties

This example introduces the module deeptrack.properties.

## What is a property?

Each feature (instance of the class `Feature`, see [features_example](features_example.ipynb)) can have several properties (instances of the class `Property`).
A propety has a value accessible through the field `current_value`, whose data type is not restricted. 
This value is updated through a sampling rule (method `.update()`), which is passed to the class constructor on initialization. 

## What is a sampling rule?

The sampling rule determines how the value of a property is updated upon calling `.update()`.
A sampling rule is defined when an instance of the class Property is created and can be of any type. 
When calling `.update()`, the value of the property is updated according to the first of the following that applies:
    
1.  If the sampling rule has a method `.sample()`, call `.sample()` and return the output.
2.  If the sampling rule is a ``dict``, sample each value and combine the result into a new ``dict`` using the original keys. 
3.  If the sampling rule is a ``list``, sample each element of the list and combine the result into a new ``list``.
4.  If the sampling rule is an iterable, return the next value. If the iterable is exhausted, the value of the property is not changed.
5.  If the sampling rule is callable, call it and return the result.
6.  If none of the above apply, return the sampling rule itself.

In [20]:
import numpy as np
from deeptrack.properties import Property, SequentialProperty, PropertyDict

## 1.1 Property with a constant value

The simplest example of a property is one that does not change during an update call.
This is commonly either a number or a tuple, but can be any data type that will be evaluated by case 6.
If you want to have a constant property with a value that would be evaluated by cases 1-5 (e.g., a list or a function), you can  wrap it as the output of a lambda function.

In [21]:
# NUMBER

P = Property(1)
print("The current value of the property is", P.current_value)

P.update() # Numbers are not changed after an update() call
print("The current value of the property is", P.current_value)

The current value of the property is 1
The current value of the property is 1


In [22]:
# TUPLE

P = Property((1, [2, 3], None))
print("The current value of the property is", P.current_value)

P.update() # Tuples are not changed after an update() call
print("The current value of the property is", P.current_value)

The current value of the property is (1, [2, 3], None)
The current value of the property is (1, [2, 3], None)


In [23]:
# WRAPPED LIST

P = Property(lambda: [np.random.rand, 1, {}])
print("The current value of the property is", P.current_value)

P.update() # Objects wrapped in functions are not changed after an update() call
print("The current value of the property is", P.current_value)

The current value of the property is [<built-in method rand of numpy.random.mtrand.RandomState object at 0x0000024DDCC00378>, 1, {}]
The current value of the property is [<built-in method rand of numpy.random.mtrand.RandomState object at 0x0000024DDCC00378>, 1, {}]


## 1.2 Property with a discrete random value 

Discrete randomness can be achieved by a function (case 5).

In [24]:
# FUNCTION

P = Property(lambda: np.random.randint(0, 10))
for _ in range(5): 
    P.update()
    print("The current value of the property is", P.current_value)

The current value of the property is 1
The current value of the property is 4
The current value of the property is 8
The current value of the property is 9
The current value of the property is 5


In [25]:
# BINARY CHOICE

P = Property(lambda: 1 if np.random.rand() > 0.75 else 0)
for _ in range(5): 
    P.update()
    print("The current value of the property is", P.current_value)

The current value of the property is 0
The current value of the property is 0
The current value of the property is 0
The current value of the property is 0
The current value of the property is 0


## 1.3 Property with a continuous random value 

Continuous randomness is typically achieved by passing a function that returns a coninuous random value. This function should take no input, as noted in case 5. To use a function that needs arguments, wrap it in a function that calls it with the correct arguments.

In [26]:
# FUNCTION WITH NO INPUT

P = Property(np.random.rand)
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

The current value of the property is 0.7091281896563094
The current value of the property is 0.3915972179156568
The current value of the property is 0.39638537939593776
The current value of the property is 0.05938215863327678
The current value of the property is 0.1399123253529757


In [27]:
# WRAPPED FUNCTION

P = Property(lambda: np.random.normal(1, 5))
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

The current value of the property is -0.6512569496025855
The current value of the property is 4.693888378095286
The current value of the property is 1.058907313179013
The current value of the property is -0.9157391668664014
The current value of the property is 12.098982529780779


## 1.4 Property with a deterministically changing value

Deterministically changing properties can be achieved using either an iterator (case 4) or a function (case 5). For the output of a function to change deterministically between calls, it should reference some variable outside its definition. Once an iterator has been exhausted, it will always return its last value.

In [28]:
# ITERATOR

P = Property(iter([1, 2, 3, 4, 5]))
for _ in range(10):
    P.update()
    print("The current value of the property is", P.current_value)

The current value of the property is 1
The current value of the property is 2
The current value of the property is 3
The current value of the property is 4
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5
The current value of the property is 5


In [29]:
# FUNCTION

fibbonacci = [1, 1]
def fibbonacci_sequence():
    fibbonacci.append(fibbonacci[-2] + fibbonacci[-1])
    return fibbonacci

P = Property(fibbonacci_sequence)
for _ in range(10):
    P.update()
    print("The current value of the property is", P.current_value)

The current value of the property is [1, 1, 2]
The current value of the property is [1, 1, 2, 3]
The current value of the property is [1, 1, 2, 3, 5]
The current value of the property is [1, 1, 2, 3, 5, 8]
The current value of the property is [1, 1, 2, 3, 5, 8, 13]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89]
The current value of the property is [1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144]


## 1.5 Property with dependent value

The value of a property can be dependent on the value on some other property. It does this by accepting some keyword argument corresponding to the name of the independent property. Instances of `Feature` will handle this automatically.

In [30]:
random_number = Property(lambda: np.random.rand())

def get_dependent_number(random_number):
    return random_number + 1

dependent_number = Property(get_dependent_number)

for _ in range(5):
    dependent_number.update(random_number=random_number)
    
    dependent_number.has_updated_since_last_resolve = False
    random_number.has_updated_since_last_resolve = False
    
    print("The current value of the independent property is", random_number.current_value)
    print("The current value of the dependent property is", dependent_number.current_value, "\n")

The current value of the independent property is 0.26126281870933443
The current value of the dependent property is 1.2612628187093344 

The current value of the independent property is 0.12241346110595253
The current value of the dependent property is 1.1224134611059524 

The current value of the independent property is 0.9764026503707658
The current value of the dependent property is 1.976402650370766 

The current value of the independent property is 0.00015339370945599562
The current value of the dependent property is 1.000153393709456 

The current value of the independent property is 0.17485924331893565
The current value of the dependent property is 1.1748592433189358 



## 2. What is SequentialProperty?

The class `SequentialProperty` extends `Property` to handle cases where a sequence of values are required. This is most commonly used for creating videos, where sequential properties contain the value of some property at each frame. 

Sequential properties are created and sampled similarly to standard properties. They accept a sampling rule as the first parameter, but also optionally accept an initializer through the keyword argument `initializer`. The initializer is responsible for the first step in each sequence. 

The sampling rule is sampled once per step in the sequence, and concatenated into a list of values. To facilitate the creation of time dependent properties, sampling rules that are function may additionally accept a few keyword arguments:

* `previous_value`: The value of the property at the previous step. Is `None` at the first step.
* `previous_values`: The value of the property at all previous steps as a list. Is `[]` at the first step.
* `sequence_step`: The step in the sequence being sampled.
* `sequence_length`: The length of the sequence.

## 2.1 SequentialProperty with constant value

In [31]:
sequence_length = 10
sampling_rule = 1
initializer = 0

P = SequentialProperty(sampling_rule, initializer=initializer)

for _ in range(5):
    P.update(sequence_length=sequence_length)
    
    print("The current value of the sequential property is", P.current_value)

The current value of the sequential property is [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
The current value of the sequential property is [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
The current value of the sequential property is [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
The current value of the sequential property is [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
The current value of the sequential property is [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


## 2.2 SequentialProperty with time-dependent values

In [32]:
sequence_length = 10

# a function as sampling rule
def rotation(sequence_step, sequence_length):
    return 2 * np.pi / sequence_length * sequence_step

P = SequentialProperty(rotation, initializer=0)

P.update(sequence_length=sequence_length)
print("The current value of the sequential property is", P.current_value)

The current value of the sequential property is [0.0, 0.6283185307179586, 1.2566370614359172, 1.8849555921538759, 2.5132741228718345, 3.141592653589793, 3.7699111843077517, 4.39822971502571, 5.026548245743669, 5.654866776461628]


In [33]:
sequence_length = 10

# a function as sampling rule
def random_walk(previous_value):
    return previous_value + (1 if np.random.randint(2) else -1) 

P = SequentialProperty(random_walk, initializer=0)

for _ in range(5):
    P.update(sequence_length=sequence_length)
    print("The current value of the sequential property is", P.current_value)

The current value of the sequential property is [1, 2, 1, 0, -1, -2, -1, -2, -3, -2]
The current value of the sequential property is [-1, -2, -3, -2, -1, -2, -1, -2, -3, -2]
The current value of the sequential property is [1, 0, 1, 0, -1, 0, 1, 0, 1, 0]
The current value of the sequential property is [-1, -2, -3, -2, -1, 0, 1, 2, 1, 2]
The current value of the sequential property is [1, 0, 1, 2, 1, 2, 3, 4, 5, 4]


## 2.3 SequentialProperty dependent on Property

In [34]:
sequence_length = 10

random_walk_bias = Property(lambda: np.random.rand())

def random_walk(previous_value, random_walk_bias):
    return previous_value + (1 if np.random.rand() < random_walk_bias else -1) 

P = SequentialProperty(random_walk, initializer=0)

for _ in range(5):
    P.update(sequence_length=sequence_length, 
             random_walk_bias=random_walk_bias)
    
    
    print("The current value of the bias property is", random_walk_bias.current_value)
    print("The current value of the sequential property is", P.current_value, "\n")

The current value of the bias property is 0.9181049263894205
The current value of the sequential property is [-1, 0, -1, 0, 1, 2, 1, 0, -1, 0] 

The current value of the bias property is 0.05398862748068112
The current value of the sequential property is [1, 0, 1, 0, 1, 0, -1, -2, -1, -2] 

The current value of the bias property is 0.25119264719083556
The current value of the sequential property is [-1, 0, 1, 2, 1, 2, 3, 4, 5, 4] 

The current value of the bias property is 0.3492396211338358
The current value of the sequential property is [-1, -2, -3, -2, -3, -2, -1, 0, 1, 0] 

The current value of the bias property is 0.23196448654490798
The current value of the sequential property is [-1, -2, -3, -2, -1, -2, -1, -2, -3, -4] 



## 2.4 SequentialProperty dependent on SequentialProperty

In [35]:
sequence_length = 10

step_length = SequentialProperty(lambda: np.random.poisson(1))

def random_walk(step_length, previous_value=0):
    return previous_value + (step_length if np.random.rand() < 0.5 else -step_length) 

P = SequentialProperty(random_walk)

for _ in range(5):
    P.update(sequence_length=sequence_length, 
             step_length=step_length)
    
    
    print("The current value of the step-length property is", step_length.current_value)
    print("The current value of the sequential property is", P.current_value, "\n")

The current value of the step-length property is [2, 0, 1, 0, 0, 3, 2, 0, 0, 1]
The current value of the sequential property is [0, 1, 3, 3, 4, 4, 6, 5, 4, 3] 

The current value of the step-length property is [1, 1, 2, 0, 2, 0, 1, 2, 0, 3]
The current value of the sequential property is [1, 2, 5, 6, 5, 4, 1, 0, -1, 2] 

The current value of the step-length property is [1, 1, 2, 0, 0, 2, 0, 0, 0, 2]
The current value of the sequential property is [-2, -2, 1, -1, 0, 2, 3, 5, 4, 2] 

The current value of the step-length property is [1, 1, 0, 0, 2, 2, 2, 1, 0, 2]
The current value of the sequential property is [0, -1, -1, -3, -4, -5, -5, -6, -2, -4] 

The current value of the step-length property is [1, 0, 0, 0, 2, 0, 2, 0, 2, 0]
The current value of the sequential property is [-3, -3, -2, 0, 0, 0, 0, 1, 1, 1] 



## 3. What is a PropertyDict?

Another class contained in the module deeptrack.properties is `PropertyDict`. This is a dictionary of properties (keys: name of properties; values: properties) complemented by utility methods to manage collections of properties. These include:

* `.current_value_dict()`, which creates and returns a dictionary with the current value of all properties in the PropertyDict (keys: name of properties; values: current values of the properties).
* `.update()`, which calls the method `.update()` on all properties in the PropertyDict.
* `.sample()`, which calls the method `.sample()` on all properties in the PropertyDict, and creates and returns a dictionary from the output (keys: name of properties; values: sample outputs of the properties).


In [36]:
property_dict = PropertyDict(
    foo=Property(1),
    bar=Property(np.random.rand)
)

for _ in range(5):
    property_dict.update()
    print("The current values of the properties in property_dict are", property_dict.current_value_dict(is_resolving=True))

The current values of the properties in property_dict are {'foo': 1, 'bar': 0.24053683462170916}
The current values of the properties in property_dict are {'foo': 1, 'bar': 0.03691219274130386}
The current values of the properties in property_dict are {'foo': 1, 'bar': 0.8133165120113509}
The current values of the properties in property_dict are {'foo': 1, 'bar': 0.07861900028397373}
The current values of the properties in property_dict are {'foo': 1, 'bar': 0.4462205656548687}


Note that `current_value_dict()` is called with `is_resolving=True`. This makes the `PropertyDict` automatically set `has_updated_since_last_resolve` to `False` on all properties in the `PropertyDict`.  